# Open Exploration

As we are interested in how dike ring 4 could best balance its economic objectives with a sustainable flood-risk management plan, we would want to first use open exploration to understand how regions in the uncertainty space map to the whole outcome map.

Using open exploration, we will systematically sample the uncertainty or decision space.

## Running the model

Importing all necessary libraries, checking that libraries are up-to-date, and choosing problem formulation.

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [3]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.0.3
2.4


In [4]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [5]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(5)

In [6]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('discount rate 3', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132)
RealParameter('A.1_Bmax', 30, 350)
RealParameter('A.1_pfail', 0, 1)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350)
RealParameter('A.2_pfail', 0, 1)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350)
RealParameter('A.3_pfail', 0, 1)
CategoricalParameter('A.3_Brate', [0, 1, 2])
RealParameter('A.4_Bmax', 30, 350)
RealParameter('A.4_pfail', 0, 1)
CategoricalParameter('A.4_Brate', [0, 1, 2])
RealParameter('A.5_Bmax', 30, 350)
RealParameter('A.5_pfail', 0, 1)
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [7]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1)
IntegerParameter('0_RfR 1', 0, 1)
IntegerParameter('0_RfR 2', 0, 1)
IntegerParameter('0_RfR 3', 0, 1)
IntegerParameter('1_RfR 0', 0, 1)
IntegerParameter('1_RfR 1', 0, 1)
IntegerParameter('1_RfR 2', 0, 1)
IntegerParameter('1_RfR 3', 0, 1)
IntegerParameter('2_RfR 0', 0, 1)
IntegerParameter('2_RfR 1', 0, 1)
IntegerParameter('2_RfR 2', 0, 1)
IntegerParameter('2_RfR 3', 0, 1)
IntegerParameter('3_RfR 0', 0, 1)
IntegerParameter('3_RfR 1', 0, 1)
IntegerParameter('3_RfR 2', 0, 1)
IntegerParameter('3_RfR 3', 0, 1)
IntegerParameter('4_RfR 0', 0, 1)
IntegerParameter('4_RfR 1', 0, 1)
IntegerParameter('4_RfR 2', 0, 1)
IntegerParameter('4_RfR 3', 0, 1)
IntegerParameter('EWS_DaysToThreat', 0, 4)
IntegerParameter('A.1_DikeIncrease 0', 0, 10)
IntegerParameter('A.1_DikeIncrease 1', 0, 10)
IntegerParameter('A.1_DikeIncrease 2', 0, 10)
IntegerParameter('A.1_DikeIncrease 3', 0, 10)
IntegerParameter('A.2_DikeIncrease 0', 0, 10)
IntegerParameter('A.2_DikeIncrease 1', 0, 10)
I

In [8]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1_Expected Annual Damage 0')
ScalarOutcome('A.1_Dike Investment Costs 0')
ScalarOutcome('A.1_Expected Number of Deaths 0')
ScalarOutcome('A.2_Expected Annual Damage 0')
ScalarOutcome('A.2_Dike Investment Costs 0')
ScalarOutcome('A.2_Expected Number of Deaths 0')
ScalarOutcome('A.3_Expected Annual Damage 0')
ScalarOutcome('A.3_Dike Investment Costs 0')
ScalarOutcome('A.3_Expected Number of Deaths 0')
ScalarOutcome('A.4_Expected Annual Damage 0')
ScalarOutcome('A.4_Dike Investment Costs 0')
ScalarOutcome('A.4_Expected Number of Deaths 0')
ScalarOutcome('A.5_Expected Annual Damage 0')
ScalarOutcome('A.5_Dike Investment Costs 0')
ScalarOutcome('A.5_Expected Number of Deaths 0')
ScalarOutcome('RfR Total Costs 0')
ScalarOutcome('Expected Evacuation Costs 0')
ScalarOutcome('A.1_Expected Annual Damage 1')
ScalarOutcome('A.1_Dike Investment Costs 1')
ScalarOutcome('A.1_Expected Number of Deaths 1')
ScalarOutcome('A.2_Expected Annual Damage 1')
ScalarOutcome('A.2_Dike Investment

In [9]:
#running the model through EMA workbench
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with MultiprocessingEvaluator(dike_model) as evaluator:
    experiments,outcomes = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 60 cases completed
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 120 cases completed
[MainProcess/INFO] 140 cases completed
[MainProcess/INFO] 160 cases completed
[MainProcess/INFO] 180 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [10]:
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 2,A.4_DikeIncrease 3,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,A.5_DikeIncrease 3,EWS_DaysToThreat,scenario,policy,model
0,66.0,71.866583,1.5,0.508006,110.170273,1,0.754108,90.794921,1,0.671163,...,5.0,4.0,5.0,4.0,6.0,6.0,0.0,0,0,dikesnet
1,60.0,111.164455,1,0.623435,331.519670,1,0.020332,222.720907,1.5,0.010192,...,5.0,4.0,5.0,4.0,6.0,6.0,0.0,1,0,dikesnet
2,46.0,103.473107,1.5,0.492834,154.173929,1,0.429348,115.482931,1,0.063180,...,5.0,4.0,5.0,4.0,6.0,6.0,0.0,2,0,dikesnet
3,116.0,160.470702,10,0.008957,159.563340,1.5,0.970460,291.133501,10,0.726318,...,5.0,4.0,5.0,4.0,6.0,6.0,0.0,3,0,dikesnet
4,101.0,232.406429,1,0.965338,97.069890,1.5,0.532433,51.748216,1,0.142387,...,5.0,4.0,5.0,4.0,6.0,6.0,0.0,4,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,84.0,137.663323,10,0.679594,47.720097,10,0.270682,174.118103,1.5,0.653429,...,5.0,8.0,4.0,1.0,9.0,4.0,2.0,45,3,dikesnet
196,120.0,145.414413,1,0.594603,165.071044,10,0.984962,66.865696,1,0.503989,...,5.0,8.0,4.0,1.0,9.0,4.0,2.0,46,3,dikesnet
197,111.0,89.666320,1.5,0.431414,286.221766,1.5,0.803602,68.781215,1.5,0.289000,...,5.0,8.0,4.0,1.0,9.0,4.0,2.0,47,3,dikesnet
198,12.0,32.764512,1,0.188196,284.509027,10,0.139080,60.259673,10,0.688121,...,5.0,8.0,4.0,1.0,9.0,4.0,2.0,48,3,dikesnet


In [11]:
outcomes

{'A.1_Expected Annual Damage 0': array([       0.        ,        0.        ,        0.        ,
        58429093.69947523,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,  7982836.74650693,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.        ,
               0.        ,        0.        ,        0.   

In [12]:
import pandas as pd
outcomes_new= {k:v.tolist() for k, v in outcomes.items()}

In [13]:
outcomes_new

{'A.1_Expected Annual Damage 0': [0.0,
  0.0,
  0.0,
  58429093.69947523,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  7982836.746506935,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  

In [14]:
outcome_df = pd.DataFrame.from_dict(outcomes_new)
outcome_df.head()

,A.1_Expected Annual Damage 0,A.1_Dike Investment Costs 0,A.1_Expected Number of Deaths 0,A.2_Expected Annual Damage 0,A.2_Dike Investment Costs 0,A.2_Expected Number of Deaths 0,A.3_Expected Annual Damage 0,A.3_Dike Investment Costs 0,A.3_Expected Number of Deaths 0,A.4_Expected Annual Damage 0,...,A.3_Dike Investment Costs 3,A.3_Expected Number of Deaths 3,A.4_Expected Annual Damage 3,A.4_Dike Investment Costs 3,A.4_Expected Number of Deaths 3,A.5_Expected Annual Damage 3,A.5_Dike Investment Costs 3,A.5_Expected Number of Deaths 3,RfR Total Costs 3,Expected Evacuation Costs 3
0,0.000000e+00,4.229151e+07,0.000000,3.314961e+05,3.824512e+07,0.000625,0.0,4.060905e+07,0.0,4.932913e+06,...,5.873534e+07,0.0,0.0,1.604874e+07,0.0,0.0,6.666619e+07,0.0,205800000.0,0.0
1,0.000000e+00,4.229151e+07,0.000000,2.151562e+08,3.824512e+07,0.269409,0.0,4.060905e+07,0.0,0.000000e+00,...,5.873534e+07,0.0,0.0,1.604874e+07,0.0,0.0,6.666619e+07,0.0,205800000.0,0.0
2,0.000000e+00,4.229151e+07,0.000000,5.622466e+06,3.824512e+07,0.007072,0.0,4.060905e+07,0.0,0.000000e+00,...,5.873534e+07,0.0,0.0,1.604874e+07,0.0,0.0,6.666619e+07,0.0,205800000.0,0.0
3,5.842909e+07,4.229151e+07,0.071501,0.000000e+00,3.824512e+07,0.000000,0.0,4.060905e+07,0.0,0.000000e+00,...,5.873534e+07,0.0,0.0,1.604874e+07,0.0,0.0,6.666619e+07,0.0,205800000.0,0.0
4,0.000000e+00,4.229151e+07,0.000000,4.330319e+06,3.824512e+07,0.007149,0.0,4.060905e+07,0.0,0.000000e+00,...,5.873534e+07,0.0,0.0,1.604874e+07,0.0,0.0,6.666619e+07,0.0,205800000.0,0.0


In [15]:
#drop outcomes that are not of interest 
outcomes_of_interest = ['A.4_Expected Annual Damage 0','A.4_Dike Investment Costs 0','A.4_Expected Number of Deaths 0',
                        'A.4_Expected Annual Damage 1','A.4_Dike Investment Costs 1','A.4_Expected Number of Deaths 1',
                        'A.4_Expected Annual Damage 2','A.4_Dike Investment Costs 2','A.4_Expected Number of Deaths 2',
                        'A.4_Expected Annual Damage 3','A.4_Dike Investment Costs 3','A.4_Expected Number of Deaths 3']

In [16]:
outcome_df = outcome_df[outcomes_of_interest]
outcome_df.head()

,A.4_Expected Annual Damage 0,A.4_Dike Investment Costs 0,A.4_Expected Number of Deaths 0,A.4_Expected Annual Damage 1,A.4_Dike Investment Costs 1,A.4_Expected Number of Deaths 1,A.4_Expected Annual Damage 2,A.4_Dike Investment Costs 2,A.4_Expected Number of Deaths 2,A.4_Expected Annual Damage 3,A.4_Dike Investment Costs 3,A.4_Expected Number of Deaths 3
0,4.932913e+06,7.332080e+06,0.003841,0.0,1.722864e+07,0.0,0.0,1.548896e+07,0.0,0.0,1.604874e+07,0.0
1,0.000000e+00,7.332080e+06,0.000000,0.0,1.722864e+07,0.0,0.0,1.548896e+07,0.0,0.0,1.604874e+07,0.0
2,0.000000e+00,7.332080e+06,0.000000,0.0,1.722864e+07,0.0,0.0,1.548896e+07,0.0,0.0,1.604874e+07,0.0
3,0.000000e+00,7.332080e+06,0.000000,0.0,1.722864e+07,0.0,0.0,1.548896e+07,0.0,0.0,1.604874e+07,0.0
4,0.000000e+00,7.332080e+06,0.000000,0.0,1.722864e+07,0.0,0.0,1.548896e+07,0.0,0.0,1.604874e+07,0.0


## Visual Analysis

To find out how the outcomes relate to each other, we will create a pair plot with all the total outcomes.

In [17]:
sns.pairplot(outcomes_df)

[MainProcess/INFO] no time dimension found in results


KeyboardInterrupt: 

Since we 

In [ ]:
experiments, outcomes = results
policies = experiments['policy']

data = pd.DataFrame.from_dict(outcomes)
data['policy'] = policies

sns.pairplot(data, hue='policy', vars=outcomes.keys(), diag_kind='hist')
plt.show()